# OSIRIS-REx: size analysis of sample OREX-800107-0
OREX-800107-0 is a 6.4-gram aggregate sample that contains approximately 1.5 million grains.

In [ ]:
# import modules
import numpy as np
import matplotlib.pyplot as plt
import skimage                            # used for image processing
from scipy import ndimage                 # used to label all the grains
import os                                 # used to interact with operating systems
from scipy.spatial.distance import pdist  # pdist = pairwise distances
from scipy.ndimage import find_objects
from tabulate import tabulate
from numba import jit, njit, prange
from scipy.optimize import curve_fit      # used for power law plotting
import pandas as pd

In [ ]:
# assign variables to outputs from Ilastic

# change for a different computer or a different Ilastik stack:

# base_dir = 'C:\\Users\\savan'
# folder = '\\107IlastikSample\\'
# images = '\\107IlastikSample_{i:04d}.tif'

# initialize an np.ndarray for data
# NOTE: data may be interpreted in y, x, z
cube = np.zeros((1384, 1390, 1751), dtype='int8')
# loop through every slice in the xy plane
for i in range(0,1719):
    # data = open(os.path.join('C:', os.sep, 'Users', 'savan', '107IlastikSample', '107IlastikSample_{i:04d}.tif')).read()
    # define a path to the Ilastik output
    data = r'C:\\Users\\savan\\107IlastikSample' + f'/107IlastikSample_{i:04d}' + '.tif'  # address for laptop
    # assign data to an np.ndarray
    cube[:,:,i] = plt.imread(data)
    # progress check:
    if i % 10 == 0:
       print(i, end='\r')
print('done')

In [ ]:
# determine the voxel count of every class
print(f'Phase 1 (particle): {np.sum(cube==1): 0.2g} voxels')
print(f'Phase 2 (matrix): {np.sum(cube==2): 0.2g} voxels')
print(f'Phase 3 (vessel): {np.sum(cube==3): 0.2g} voxels')

Grainmask evaluation:
--------------------

In [ ]:
# form a grain mask based on the stored grains
grainmask = (cube == 1).astype('uint8')

# plot the grain mask at different z-slices:
# can plot more of these to see how accurate Ilastik was in comparison to raw data
# 0 represents matrix and 1 represents grain

# these masks allow us to quickly analyze the accuracy of the Ilastik model
# are the individual grains clear from these grain masks? 
# are there prominent places where grains have not been properly segmented?

plt.figure('Grain Mask(400)', figsize=(15,15))
plt.title('Grain Mask at z = 400')
plt.imshow(grainmask[600:800,600:800,400])
plt.colorbar()
plt.show()

plt.figure('Grain Mask (700)', figsize=(15,15))
plt.title('Grain Mask at z = 700')
plt.imshow(grainmask[600:800,600:800,700])
plt.colorbar()
plt.show()

plt.figure('Grain Mask (1000)', figsize=(15,15))
plt.title('Grain Mask at z = 1000')
plt.imshow(grainmask[600:800,600:800,1000])
plt.colorbar()
plt.show()

plt.figure('Grain Mask(1300)', figsize=(15,15))
plt.title('Grain Mask at z = 1300', fontsize=30)
plt.imshow(grainmask[600:800,600:800,1300])
plt.colorbar()
plt.savefig('GrainMask1300.png', dpi=300)
plt.show()


Using the grain mask in further variable definition:
---------------------------------------------------
The GrainLabelCube is an np.ndarray that will be used in all the following codes. This cube represents the 3D data set as segmented grains.

In [ ]:
# determine the number of grains in the entire sample by counting individual islands
# needs a lot of memory to run due to extremely large data set
GrainLabelCube, NumLabel = ndimage.label(grainmask, None, output=np.uint32)
print(f'there are {NumLabel} particles that are labeled')

In [ ]:
# plot the islands in the entire sample
fig,ax = plt.subplots(1, 2, figsize=(15,15))
ax[0].imshow(grainmask[:,500,:] == 1)
ax[1].imshow(GrainLabelCube[:,500,:])
plt.show()

Compute grain volume:
---------------------

In [ ]:
@njit(parallel=True)     # using numba for speed

def ComputeVolumes(GrainLabelCube):
    '''
        This function computes the volume of every grain with units of voxels
        input:
            GrainLabelCube: np.ndarray
                stores the individual grai
        output:
            volumes: np.array
                stores the volume of every grain in GrainLabelCube
    '''

    # intialize an np.array of size of NumLabel
    volumes = np.zeros(NumLabel)
    # turn np.ndarray to a 1D array
    GrainRavel = np.ravel(GrainLabelCube)  

    # loop through every voxel in GrainLabelCube through GrainRavel
    for i in range(len(GrainRavel)):
        volumes[GrainRavel[i]] += 1
        
    # first bin is background and is bigger than any other grain
    volumes[0] = 0
    
    return volumes

In [ ]:
# assigning variable to ComputeVolumes output
volumes = ComputeVolumes(GrainLabelCube)

# Start of PSFD calculations:

Approximated longest axis calculation:
--------------------------------------

In [ ]:
# find_objects identifies and returns the bounding box coordinates for every labeled object
# returns a list of tuples where the tuples contain the coordinates
BoundingBoxes = ndimage.find_objects(GrainLabelCube)

In [ ]:
# using a bounding box APPROXIMATION for the longest axis:

def BBLongestAxis(GrainLabelCube):
    '''
        This function calculates an approximation of the longest axis of a grain 
        using a bounding box algorithm
        input:
            GrainLabelCube: np.ndarray
                stores the individual grains
        output:
            BBLongestAxes: np.ndarray
                stores an approx. of the longest axis of each grain
    '''

    # initialize an np.array to add longest axes calculations
    BBLongestAxes = np.zeros(NumLabel)

    # loop through each grain's bounding box
    for i, BoundingBox in enumerate(BoundingBoxes):

        # if bounding box is empty, move to the next bounding box
        if BoundingBox is None:
            continue

        # find the bounding box vertex coordinates
        MinCoord = np.array([BoundingBox[j].start for j in range(len(BoundingBox))])
        MaxCoord = np.array([BoundingBox[j].stop for j in range(len(BoundingBox))])

        # find the distance between the two vertex coordinates
        ApproxLongestAxis = np.linalg.norm(MaxCoord - MinCoord)

        # add the approximated axis to np.array
        # multiply value by 0.017 because each voxel is 0.017 mm
        BBLongestAxes[i] = ApproxLongestAxis * 0.017

    return BBLongestAxes

In [ ]:
# assigning variable to BBLongestAxis output
BBLongestAxes = BBLongestAxis(GrainLabelCube)

Importing the Dragonfly data from the csv file from Andy
--------------------------------------------------------

In [ ]:
# read the CSV file
DragonflyData = pd.read_csv('orex-800107-0_segmented_stats.csv', delimiter=';', encoding='utf-8')

#print(DragonflyData.head())

#print(DragonflyData.columns.tolist())

FeretMin = DragonflyData['Min Feret Diameter (mm)']
FeretMax = DragonflyData['Max Feret Diameter (mm)']

Plotting the approximation:
--------------------------
Consider cutting off grain size at roughly 5 voxel or 0.065 mm

In [ ]:
def PowerLaw(x, a, b):
    '''
        This function computes the power law of a given data set
        inputs:
            x: np.array
                the data (longest axes of particles)
            a: float
                the y-intercept of the power-law fit
            b: float
                the slope of the power-law fit
        outputs: 
            PL: float
                the power law
    '''
    
    PL = a * x ** b

    return PL

From Burke+2021:
----------------
Power-law PSFDs with the form:
\begin{equation}
    N_C = c D^{-a}
\end{equation}
where $N_C$ is the cumulative number of particles greater than or equal to $D$ per surface area unit. $D$ is the longest axis particle measurement, $a$ is the power-law index, and $c$ is the coefficient of proportionality.
- error from one PSFD came from the square root of $N_C$ which represents the Poisson detection distribution uncertainty.

From Clauset+2009:
-----------------
Power-law MLE slope for continuous data:
\begin{equation}
    \hat{\alpha} = 1 + n[\sum_{i=1}^n ln(x_i/x_{min})]^{-1}
\end{equation}
where $x_i$ are the data points greater than or equal to $x_{min}$ and $n$ is the number of those points
So, the standard error is as follows:
\begin{equation}
    \sigma = (\hat{\alpha}-1) / \sqrt{n}
\end{equation}

In [ ]:
# using information from Clauset+2009:
# note: maybe use this or continue to use the bootstrap uncertainty calc
def powerlaw_slope_and_error(x, x_min):
    x_cut = x[x >= x_min]
    n = len(x_cut)
    alpha = 1 + n / np.sum(np.log(x_cut / x_min))
    sigma = (alpha - 1) / np.sqrt(n)
    return alpha, sigma

# Plotting PSFD:

In [ ]:
# NO ERROR CALCULATION

# hard-code total analyzed volume of the sample
# used to normalize the counts
SampleVolume = 1e5   # in mm^3

# define the x-range for fitting the linear slope accurately
xmin_fit = 0.3
xmax_fit = 3.0

# create log-spaced bins for histogram plotting
bins = np.logspace(np.log10(min(BBLongestAxes)), np.log10(max(BBLongestAxes)), 10000)
counts, BinEdges = np.histogram(BBLongestAxes, bins=bins)
# determining the cumulative sum from the smallest to the largest particle
CumulativeCounts = np.cumsum(counts[::-1])[::-1] / SampleVolume  # per mm³
BinCenters = (BinEdges[:-1] + BinEdges[1:]) / 2

# ignore zeros in the counts due to loglog space
mask = CumulativeCounts > 0
x = BinCenters[mask]
y = CumulativeCounts[mask]

# mask the data to be within the selected x-range for accurate fitting
# used to find the slope only based on the portion of the plot that is linear
mask2 = (x >= xmin_fit) & (x <= xmax_fit)
x_fit = x[mask2]
y_fit = y[mask2]

# fit the curve to the dataset
parameters, _ = curve_fit(PowerLaw, x_fit, y_fit, p0=[0.12, -2])
a, b = parameters

# compute y=mx+b line for slope
x_line = np.logspace(np.log10(xmin_fit), np.log10(xmax_fit), 100)
y_line = PowerLaw(x_line, a, b)

# plot the PSFD
plt.figure(figsize=(8,6))
plt.loglog(x, y, 'o', ms=1, label='Cumulative Number\nof Particles', color='black')
# highlight portion of the data that contributes to the slope
plt.loglog(x_fit, y_fit, 'o', ms=1, color='yellowgreen')
plt.loglog(x_line, y_line, ls='--', ms=10, label=f"Slope = {b:.2f}", color='green')
# labels and organization
plt.xlabel("BB Longest Axis (mm)", fontsize=13).set_style('italic')
plt.ylabel(f"Cumulative Frequency per $mm^3$", fontsize=13).set_style('italic')
plt.title("Appromixated Particle Size Frequency Distribution\n for sample OREX-800107-0 using Ilastik", fontsize=14, weight='bold')
plt.grid(True, which='both', linestyle=':', linewidth=0.5)
plt.legend()
plt.tight_layout()
plt.savefig('PSFD107_Ilastik_approx.png')
plt.show()

In [ ]:
# CALCULATING ERROR WITH A POISSON DISTRIBUTION

# hard-code total analyzed volume of the sample
# used to normalize the counts
SampleVolume = 1e5   # in mm^3

# define the x-range for fitting the linear slope accurately
xmin_fit = 0.3
xmax_fit = 3.0

# create log-spaced bins for histogram plotting
bins = np.logspace(np.log10(min(BBLongestAxes)), np.log10(max(BBLongestAxes)), 1000)
counts, BinEdges = np.histogram(BBLongestAxes, bins=bins)
# determine the cumulative sum from the smallest to the largest particle
CumulativeCounts = np.cumsum(counts[::-1])[::-1] / SampleVolume  # per mm^3
BinCenters = (BinEdges[:-1] + BinEdges[1:]) / 2

# ignore zeros in the counts due to loglog space
mask = CumulativeCounts > 0
x = BinCenters[mask]
y = CumulativeCounts[mask]

# mask the data to be within the selected x-range for accurate fitting
# used to find the slope only based on the portion of the plot that is linear
mask2 = (x >= xmin_fit) & (x <= xmax_fit)
x_fit = x[mask2]
y_fit = y[mask2]

# fit the curve to the dataset
parameters, _ = curve_fit(PowerLaw, x_fit, y_fit, p0=[0.12, -2])
a, b = parameters

# compute y=mx+b line for slope
x_line = np.logspace(np.log10(xmin_fit), np.log10(xmax_fit), 100)
y_line = PowerLaw(x_line, a, b)

# compute standard deviation of cumulative counts using poisson distribution uncertainty
errors = np.sqrt(y) / SampleVolume  # SD per mm^3
# compute log-scale error (for log-log plots)
# note: when this is plotted with the PSFD, the plot is incorrect... 
# unsure if this is due to a math error or a data error...
log_errors = (1 / np.log(10)) * (1 / np.sqrt(y))  

# plot the PSFD
plt.figure(figsize=(8,6))
plt.loglog(x, y, 'o', ms=1, label='Cumulative Number\nof Particles', color='black')
# highlight portion of the data that contributes to the slope
plt.loglog(x_fit, y_fit, 'o', ms=1, color='yellowgreen')
plt.loglog(x_line, y_line, ls='--', ms=10, label=f"Slope = {b:.2f}", color='green')

# plot error bars on cumulative counts
# corresponds to uncertainty in the cumulative frequency
plt.errorbar(x, y, yerr=errors, fmt='none', ecolor='gray', alpha=0.3, capsize=1, label='Error')

# labels and organization
plt.xlabel("BB Longest Axis (mm)", fontsize=13).set_style('italic')
plt.ylabel(f"Cumulative Frequency per $mm^3$", fontsize=13).set_style('italic')
plt.title("Appromixated Particle Size Frequency Distribution\n for sample OREX-800107-0 using Ilastik", fontsize=14, weight='bold')
plt.grid(True, which='both', linestyle=':', linewidth=0.5)
plt.legend()
plt.tight_layout()
plt.savefig('PSFD107_Ilastik_approx_errors.png', dpi=300)
plt.show()

In [ ]:
# CALCULATING ERROR WITH BOOTSTRAP METHOD FROM CLAUSSET ET AL.

# hard-code total volume of the sample (approximated)
SampleVolume = 1e5  # in mm^3

# define the x-range for fitting the linear slope accurately
xmin_fit = 0.3
xmax_fit = 3.0

# create log-spaced bins for histogram plotting
bins = np.logspace(np.log10(min(BBLongestAxes)), np.log10(max(BBLongestAxes)), 10000)
BinEdges, _ = np.histogram(BBLongestAxes, bins=bins)
# determine the cumulative sum from the smallest to the largest particle
CumulativeCounts = np.cumsum(BinEdges[::-1])[::-1] / SampleVolume  # per mm³
BinCenters = (bins[:-1] + bins[1:]) / 2

# remove zero counts to avoid log errors
mask = CumulativeCounts > 0
x = BinCenters[mask]
y = CumulativeCounts[mask]

# mask the values for the slope to those previously defined
mask2 = (x >= xmin_fit) & (x <= xmax_fit)
x_fit = x[mask2]
y_fit = y[mask2]

# fit original data to get initial slope
parameteres, _ = curve_fit(PowerLaw, x_fit, y_fit, p0=[0.1, -2.1])
a_fit, b_fit = parameters

# compute the slope line for plotting
x_line = np.logspace(np.log10(xmin_fit), np.log10(xmax_fit), 100)
y_line = PowerLaw(x_line, a_fit, b_fit)

# loop through slope values to find uncertainty
B = 1000  # use at least 5000 for jounral accuracy
BootstrapSlopes = []

for i in range(B):
    # create a randomized data set
    # less biased towards one part of the data
    sample = np.random.choice(BBLongestAxes, size=len(BBLongestAxes), replace=True)
    
    # compute histogram and cumulative counts with randomized data
    bins = np.logspace(np.log10(min(sample)), np.log10(max(sample)), 10000)
    BinEdges, i = np.histogram(sample, bins=bins)
    CumulativeCounts = np.cumsum(BinEdges[::-1])[::-1] / SampleVolume
    BinCenters = (bins[:-1] + bins[1:]) / 2

    # ensure there are no zero counts for log space
    mask3 = CumulativeCounts > 0
    x_bootstrap = BinCenters[mask3]
    y_bootstrap = CumulativeCounts[mask3]

    # ensure the x and y values are within the ranges for best-fit
    mask4 = (x_bootstrap >= xmin_fit) & (x_bootstrap <= xmax_fit)
    x_fit_bootstrap = x_bootstrap[mask4]
    y_fit_bootstrap = y_bootstrap[mask4]

    # try to fit the above power law to the bootstrap data
    try:
        parameters_bootstrap, _ = curve_fit(PowerLaw, x_fit_bootstrap, y_fit_bootstrap, p0=[0.1, -2.1])
        a_bootstrap, b_bootstrap = parameters_bootstrap
        BootstrapSlopes.append(b_bootstrap)
    # if the fit fails, move to the next bootstrap value
    except:
        continue 

# convert list to np array
BootstrapSlopes = np.array(BootstrapSlopes)
mean_slope = np.mean(BootstrapSlopes)
# define confidence intervals
# will determine the +/- error for the slope
confidence_lower = np.percentile(BootstrapSlopes, 2.5)  # can change these percentiles
confidence_upper = np.percentile(BootstrapSlopes, 97.5)

# plot the PSFD
plt.figure(figsize=(8, 6))
plt.loglog(x, y, '.', ms=1, label='Cumulative Number\nof Particles', color='black')
# highlight the region of the data that corresponds to the best-fit slope
plt.loglog(x_fit, y_fit, 'o', ms=1, color='yellowgreen')

# plot best fit slope with bootstrap uncertainty
plt.loglog(x_line, y_line, '--', ms=5, 
           label=f"Slope = {b_fit:.2f}\n95% confidence interval: [{confidence_lower:.2f}, {confidence_upper:.2f}]", color='green')

# labels and organization
plt.xlabel('BB Longest Axis (mm)', fontsize=13).set_style('italic')
plt.ylabel('Cumulative Frequency per mm$^3$', fontsize=13).set_style('italic')
plt.title("Approximated Particle Size Frequency Distribution\nfor Sample OREX-800107-0 Using Ilastik", fontsize=14, weight='bold')
plt.grid(which='both', linestyle=':', linewidth=0.5)
plt.legend()
plt.tight_layout()
plt.savefig('PSFD107_Ilastik_bootstrap.png')
plt.show()

In [ ]:
# make an approximated particle number table

# define bins for the particle size range
bins = [0.01, 0.02, 0.03, 0.05, 0.07, 0.1, 0.2, 0.5, 0.8, 1.0, 2.0, 3.0, 5.0, 7.0]  
labels = [f"{bins[i]}-{bins[i+1]}" for i in range(len(bins) - 1)]

# count the number of particles in each bin
counts, _ = np.histogram(BBLongestAxes, bins=bins)

# create table data
table = list(zip(labels, counts))

print(tabulate(table, headers=["Size Range (mm)", "Number of Particles"], tablefmt="simple"))

Plotting using Dragonfly:
-------------------------

In [ ]:
# NO ERROR CALCULATION

# hard-code total analyzed volume of the sample
# used to normalize the counts
SampleVolume = 1e5   # in mm^3

# define the x-range for fitting the linear slope accurately
xmin_fit = 0.5
xmax_fit = 2.0

# create log-spaced bins for histogram plotting
bins = np.logspace(np.log10(min(FeretMax)), np.log10(max(FeretMax)), 10000)
counts, BinEdges = np.histogram(FeretMax, bins=bins)
# determining the cumulative sum from the smallest to the largest particle
CumulativeCounts = np.cumsum(counts[::-1])[::-1] / SampleVolume  # per mm³
BinCenters = (BinEdges[:-1] + BinEdges[1:]) / 2

# ignore zeros in the counts due to loglog space
mask = CumulativeCounts > 0
x = BinCenters[mask]
y = CumulativeCounts[mask]

# mask the data to be within the selected x-range for accurate fitting
# used to find the slope only based on the portion of the plot that is linear
mask2 = (x >= xmin_fit) & (x <= xmax_fit)
x_fit = x[mask2]
y_fit = y[mask2]

# fit the curve to the dataset
parameters, _ = curve_fit(PowerLaw, x_fit, y_fit, p0=[0.12, -2])
a, b = parameters

# compute y=mx+b line for slope
x_line = np.logspace(np.log10(xmin_fit), np.log10(xmax_fit), 100)
y_line = PowerLaw(x_line, a, b)

# plot the PSFD
plt.figure(figsize=(8,6))
plt.loglog(x, y, 'o', ms=1, label='Cumulative Number\nof Particles', color='black')
# highlight portion of the data that contributes to the slope
plt.loglog(x_fit, y_fit, 'o', ms=1, color='turquoise')
plt.loglog(x_line, y_line, ls='--', ms=10, label=f"Slope = {b:.2f}", color='blue')
# labels and organization
plt.xlabel("Maximum Feret Diameter (mm)", fontsize=13).set_style('italic')
plt.ylabel(f"Cumulative Frequency per $mm^3$", fontsize=13).set_style('italic')
plt.title("Particle Size Frequency Distribution\n for sample OREX-800107-0 using Dragonfly", fontsize=14, weight='bold')
plt.grid(True, which='both', linestyle=':', linewidth=0.5)
plt.legend()
plt.tight_layout()
plt.show()
plt.savefig('PSFD107_Dragonfly.png')

In [ ]:
# CALCULATING ERROR USING POSSION DISTRIBUTION

# hard-code total analyzed volume of the sample
# used to normalize the counts
SampleVolume = 1e5   # in mm^3

# define the x-range for fitting the linear slope accurately
xmin_fit = 0.5
xmax_fit = 2.0

# create log-spaced bins for histogram plotting
bins = np.logspace(np.log10(min(FeretMax)), np.log10(max(FeretMax)), 10000)
counts, BinEdges = np.histogram(FeretMax, bins=bins)
# determining the cumulative sum from the smallest to the largest particle
CumulativeCounts = np.cumsum(counts[::-1])[::-1] / SampleVolume  # per mm³
BinCenters = (BinEdges[:-1] + BinEdges[1:]) / 2

# ignore zeros in the counts due to loglog space
mask = CumulativeCounts > 0
x = BinCenters[mask]
y = CumulativeCounts[mask]

# mask the data to be within the selected x-range for accurate fitting
# used to find the slope only based on the portion of the plot that is linear
mask2 = (x >= xmin_fit) & (x <= xmax_fit)
x_fit = x[mask2]
y_fit = y[mask2]

# fit the curve to the dataset
parameters, _ = curve_fit(PowerLaw, x_fit, y_fit, p0=[0.12, -2])
a, b = parameters

# compute y=mx+b line for slope
x_line = np.logspace(np.log10(xmin_fit), np.log10(xmax_fit), 100)
y_line = PowerLaw(x_line, a, b)

# compute standard deviation of cumulative counts
errors = np.sqrt(y) / SampleVolume  # SD per mm^3
# compute log-scale error
# note: same plotting issue occurs as with Ilastik model...
log_errors = (1 / np.log(10)) * (1 / np.sqrt(y))

# plot the PSFD
plt.figure(figsize=(8,6))
plt.loglog(x, y, 'o', ms=1, label='Cumulative Number\nof Particles', color='black')
# highlight portion of the data that contributes to the slope
plt.loglog(x_fit, y_fit, 'o', ms=1, color='turquoise')
plt.loglog(x_line, y_line, ls='--', ms=10, label=f"Slope = {b:.2f}", color='blue')

# plot error bars on cumulative counts
plt.errorbar(x, y, yerr=errors, fmt='none', ecolor='gray', alpha=0.2, capsize=1, label='Error')

# labels and organization
plt.xlabel("Maximum Feret Diameter (mm)", fontsize=13).set_style('italic')
plt.ylabel(f"Cumulative Frequency per $mm^3$", fontsize=13).set_style('italic')
plt.title("Particle Size Frequency Distribution\n for sample OREX-800107-0 using Dragonfly", fontsize=14, weight='bold')
plt.grid(True, which='both', linestyle=':', linewidth=0.5)
plt.legend()
plt.tight_layout()
plt.savefig('PSFD107_Dragonfly_errors.png', dpi=300)
plt.show()

In [ ]:
# make an approximated particle number table for Dragonfly

# define bins for the particle size range
# 0.01 mm is roughly 1 voxel
# 0.151 mm is the size of the smallest particle in the Dragonfly dataset
bins = [0.01, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.5, 2.0, 3.0, 5.0, 7.0]  
labels = [f"{bins[i]}-{bins[i+1]}" for i in range(len(bins) - 1)]

# count the number of particles in each bin
counts, _ = np.histogram(FeretMax, bins=bins)

# create table data
table = list(zip(labels, counts))

print(tabulate(table, headers=["Size Range (mm)", "Number of Particles"], tablefmt="simple"))

Actual longest grain calculation:
--------------------------------

In [ ]:
# counting the grain voxels within a bounding box (NOT an approximation):
# note: requires even more memory than labeling every particle

def LongestAxis(GrainLabelCube):
    
    # initialize an array to add largest axes calculations
    LongestAxes = np.zeros(NumLabel)
    
    # loop through each bounding box region
    for i, BoundingBox in enumerate(BoundingBoxes):
    
        # if bounding box is empty, move to the next bounding box
        if BoundingBox is None:
            continue
            
        # define the region of the bounding box
        BoundedRegion = GrainLabelCube[BoundingBox]
        # find the grain coordinates inside of the bounding box
        GrainCoord = np.nonzero(BoundedRegion)
        '''
        # if no voxels are found, move to next grain
        if GrainCoord.shape[0] == 0:
            continue
        '''
        # determine the pairwise distances between every grain voxel in the bounding box
        VoxelDistances = pdist(GrainCoord, metric='euclidean')

        # loop through every pairwise distance found
        if VoxelDistances.size > 0:  # ensures that there are distances
            LongestAxes[i] = np.max(VoxelDistances)  # adds the largest distance to the list of longest axes
    
    return LongestAxes
    

In [ ]:
LongestAxes = LongestAxis(GrainLabelCube)

In [ ]:
# plot PSFD

In [ ]:
# make particle number table

# define bins for the particle size range
bins = [0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1.0, 2.0, 3.0, 5.0, 7.0]  
labels = [f"{bins[i]}-{bins[i+1]}" for i in range(len(bins) - 1)]

# count the number of particles in each bin
counts, _ = np.histogram(LongestAxes, bins=bins)

# create table data
table = list(zip(labels, counts))

print(tabulate(table, headers=["Size Range (mm)", "Number of Particles"], tablefmt="simple"))

# Determining the Shape Characteristics

For sphericity:
---------------
Use Krumbein's formula for a 3D bounding box:
\begin{equation}
\Psi = (bc/(a^2))^{1/3}
\end{equation}
where a, b, and c are the longest, intermediate, and shortest axes, respectively. If $\Psi$ is 0, the grain is angular and if $\Psi$ is 1, the grain is a perfect sphere.

In [ ]:
# note: wouldn't need this function if we don't use the bounding boxes for PSFD
# inputs of the sphericity formula are the actual axes
def GetBBDimensions(BoundingBoxes):
    '''
    '''

In [ ]:
def Sphericity(a, b, c):
    '''
        This function computes the sphericity of a particle from its dimensions
        inputs:
            a: 'float'
                longest particle axis
            b: 'float'
                intermediate particle axis
            c: 'float'
                smallest particle axis
        output:
            Sphericity: 'float'
                variable representing the sphericity of the grain
                0 for angular, 1 for perfect sphere
        
    '''

    # make sure the axes are in decreasing length
    a, b, c = sorted([a,b,c], reverse=True)

    # calculating the numerator and denominator
    num = b * c
    den = a ** 2

    Sphericity = (num/den) ** (1/3)

    return Sphericity